# DD2424 Deep Learning in Data Science
## Assignment 4

### Imports

In [1]:
# import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install tqdm

In [2]:
# import numpy as np
from data import *
from utils import *
from rnn_ import RNN

### Load data

In [3]:
data = load_and_process_text("goblet_book.txt")
book_data = data['book_data']
char_2_indices = data['char_2_indices']
indices_2_char = data['indices_2_char']

## Exercise 1
Implement and train a vanilla RNN

##### Synthesize text

In [4]:
rnn = RNN(80, 80, hidden_size=10, char_2_indices=char_2_indices, indices_2_char=indices_2_char)
output = rnn.generate()
output

"üshqL\ni'IMqRUOo4HHCQ1')yj"

##### Implement forward pass

In [5]:
seq_length = 25

x_chars = index_2_one_hot(np.array(chars_2_indices(book_data[:seq_length], char_2_indices)), 80)
y_chars = index_2_one_hot(np.array(chars_2_indices(book_data[1:seq_length + 1], char_2_indices)), 80)

In [6]:
rnn.forward(x_chars)

In [7]:
gradients = rnn.compute_gradients(x_chars, y_chars)

In [8]:
gradients_num = rnn.compute_gradients_num(x_chars, y_chars)

In [9]:
for grad in gradients:
    print(gradients[grad].shape, gradients_num[grad].shape)

(10, 10) (10, 10)
(10, 80) (10, 80)
(10, 1) (10, 1)
(80, 10) (80, 10)
(80, 1) (80, 1)


In [10]:
equal = True
threshold = 0

for grad in ['V']:
    equal = equal and np.allclose(gradients[grad], gradients_num[grad], rtol=1e-6, atol=1e-6)    
    threshold = max(threshold, np.max(np.abs(gradients[grad] - gradients_num[grad])))

print(equal)
print(threshold)

False
0.556345281902822


In [11]:
gradients['V']

array([[ 3.89961490e-04, -1.00632975e-04,  5.06463347e-04,
         1.91144547e-03, -8.94880279e-05, -2.00416739e-03,
        -3.44249846e-04,  6.38986386e-04,  1.91644681e-05,
        -9.63092378e-04],
       [ 3.89518257e-04, -1.00603973e-04,  5.07813240e-04,
         1.90931154e-03, -9.02010371e-05, -2.00195592e-03,
        -3.43138185e-04,  6.37993737e-04,  1.90333191e-05,
        -9.62269322e-04],
       [ 3.89649519e-04, -1.00420739e-04,  5.07638253e-04,
         1.90933393e-03, -9.02476038e-05, -2.00106553e-03,
        -3.42978022e-04,  6.37673446e-04,  1.88139928e-05,
        -9.61869036e-04],
       [-1.31209581e-02,  8.58993462e-03, -4.37016067e-03,
        -2.59504165e-02, -1.05003352e-02,  9.79541358e-03,
         7.63089772e-03, -1.87625182e-03, -2.03969380e-03,
         5.43816899e-04],
       [ 1.21845239e-02, -6.66212862e-03, -1.99686181e-02,
         1.35051722e-02, -1.96612053e-03, -1.21677095e-02,
         3.85019224e-03, -1.36577721e-02,  9.47473976e-03,
         9.